<a href="https://colab.research.google.com/github/jasmine400/recommender/blob/main/fast_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fastapi nest-asyncio pyngrok uvicorn

     |████████████████████████████████| 54 kB 1.7 MB/s 
     |████████████████████████████████| 745 kB 10.7 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 58 kB 5.5 MB/s 
     |████████████████████████████████| 10.9 MB 34.3 MB/s 
     |████████████████████████████████| 79 kB 7.3 MB/s 
     |████████████████████████████████| 58 kB 5.8 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=590a0caaf9c733c549353a67002f151c818d86efd76f661a17c84b90d84cd3bd
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


#original

In [ ]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn

app = FastAPI()

@app.get('/index')
async def home():
  return "Hello World"

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [60]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: http://3c58-34-136-62-38.ngrok.io


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [60]


my project dependecies

In [ ]:
!pip install -qU sentence-transformers
!pip install -qU wikipedia-api
!pip install -qU hazm
!pip install -qU clean-text[gpl]
!pip install -qU emoji

     |████████████████████████████████| 79 kB 2.8 MB/s 
     |████████████████████████████████| 3.8 MB 13.2 MB/s 
     |████████████████████████████████| 1.2 MB 59.2 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 6.5 MB 33.9 MB/s 
     |████████████████████████████████| 895 kB 58.7 MB/s 
     |████████████████████████████████| 596 kB 59.0 MB/s 
     |████████████████████████████████| 316 kB 5.3 MB/s 
     |████████████████████████████████| 1.4 MB 53.8 MB/s 
     |████████████████████████████████| 233 kB 66.1 MB/s 
     |████████████████████████████████| 53 kB 1.0 MB/s 
     |████████████████████████████████| 175 kB 10.8 MB/s 
     |████████████████████████████████| 235 kB 56.4 MB/s 


In [ ]:
!mkdir resources
!wget -q "https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip" -P resources
!unzip -qq resources/resources-0.5.zip -d resources

In [ ]:
!rm -rf /content/4ccae468eb73bf6c4f4de3075ddb5336
!rm -rf /content/preproc
!rm preprocessing.py utils.py
!mkdir -p /content/preproc
!git clone https://gist.github.com/4ccae468eb73bf6c4f4de3075ddb5336.git /content/preproc/
!mv /content/preproc/* /content/
!rm -rf /content/preproc

rm: cannot remove 'preprocessing.py': No such file or directory
rm: cannot remove 'utils.py': No such file or directory
Cloning into '/content/preproc'...
remote: Enumerating objects: 7, done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 7
Unpacking objects: 100% (7/7), done.


In [ ]:
from transformers import BertConfig, BertTokenizer

from preprocessing import cleaning
from IPython import display

import numpy as np
import pandas as pd

import hazm
import requests
import time

import torch
from sentence_transformers import models, SentenceTransformer, util
from sklearn.cluster import KMeans

import pickle

from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def rtl_print(outputs, font_size="15px", n_to_br=False):
    outputs = outputs if isinstance(outputs, list) else [outputs]
    if n_to_br:
        outputs = [output.replace('\n', '<br/>') for output in outputs]

    outputs = [f'<p style="text-align: right; direction: rtl; margin-right: 10px; font-size: {font_size};">{output}</p>' for output in outputs]
    display.display(display.HTML(' '.join(outputs)))


def load_st_model(model_name_or_path):
    word_embedding_model = models.Transformer(model_name_or_path)
    pooling_model = models.Pooling(
        word_embedding_model.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True,
        pooling_mode_cls_token=False,
        pooling_mode_max_tokens=False)

    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    return model

In [ ]:
# Load the Sentence-Transformer
embedder = load_st_model('m3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens')

Downloading:   0%|          | 0.00/519 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/621M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/404 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
#confif bert model
config = BertConfig.from_pretrained(
    'm3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens', **{

    })
print(config.to_json_string())

{
  "_name_or_path": "HooshvareLab/bert-fa-base-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}



In [ ]:
#access google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#access google drive
%cd /content/gdrive/My\ Drive/parsa_clusters
!ls

/content/gdrive/My Drive/parsa_clusters
0.csv	    146.csv	192.csv     238.csv	284.csv     5.csv
0.pickle    146.pickle	192.pickle  238.pickle	284.pickle  5.pickle
100.csv     147.csv	193.csv     239.csv	285.csv     60.csv
100.pickle  147.pickle	193.pickle  239.pickle	285.pickle  60.pickle
101.csv     148.csv	194.csv     23.csv	286.csv     61.csv
101.pickle  148.pickle	194.pickle  23.pickle	286.pickle  61.pickle
102.csv     149.csv	195.csv     240.csv	287.csv     62.csv
102.pickle  149.pickle	195.pickle  240.pickle	287.pickle  62.pickle
103.csv     14.csv	196.csv     241.csv	288.csv     63.csv
103.pickle  14.pickle	196.pickle  241.pickle	288.pickle  63.pickle
104.csv     150.csv	197.csv     242.csv	289.csv     64.csv
104.pickle  150.pickle	197.pickle  242.pickle	289.pickle  64.pickle
105.csv     151.csv	198.csv     243.csv	28.csv	    65.csv
105.pickle  151.pickle	198.pickle  243.pickle	28.pickle   65.pickle
106.csv     152.csv	199.csv     244.csv	290.csv     66.csv
106.pickle  152.pickl

tst

In [ ]:
import pandas as pd
import pickle
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cluster_df = pd.read_csv('1.csv')

pickle_in=open('1.pickle','rb')
u=pickle.load(pickle_in)


In [ ]:
print(u.shape)

(188, 768)


In [ ]:
print(u)

[[ 0.6189978  -0.32952324 -0.01990263 ... -0.39831793  1.0608387
  -0.38568023]
 [ 0.85631394 -0.23119487 -0.51203614 ... -0.5004511   0.17760974
   0.24035123]
 [ 0.6441729  -0.22126265 -0.488949   ... -0.23501642  0.12912978
  -0.15014878]
 ...
 [ 0.9066231  -0.03476292 -0.29325193 ... -0.80109084  0.4444599
  -0.1849869 ]
 [ 1.0431176  -0.3142238  -0.50786656 ... -0.54357123  0.41409647
  -0.4164911 ]
 [ 0.74692315 -0.24940388  0.12288848 ...  0.09648114  0.589434
   0.0979811 ]]


In [ ]:
print(len(u))
print(len(cluster_df))

188
188


In [ ]:
cosine_sim = cosine_similarity(u)
question_index=len(cluster_df)-1
similar_questions = list(enumerate(cosine_sim[question_index]))

In [ ]:
sorted_similar_questions = sorted(similar_questions, key=lambda x:x[1], reverse=True)
# sorted_similar_questions.pop(0)
print(type(sorted_similar_questions[0]))
if sorted_similar_questions[2][0]==187:
  sel= sorted_similar_questions[2]
  sorted_similar_questions.remove(sel)
print(sorted_similar_questions)

<class 'tuple'>
[(63, 0.99999964), (164, 0.99999964), (31, 0.90641135), (123, 0.90641135), (151, 0.90641135), (91, 0.8937838), (113, 0.8937838), (173, 0.8937838), (51, 0.892116), (146, 0.892116), (82, 0.88894266), (140, 0.88894266), (16, 0.87561524), (111, 0.87561524), (115, 0.87561524), (5, 0.87441325), (34, 0.8702492), (122, 0.8702492), (137, 0.8702492), (28, 0.8619315), (54, 0.8619315), (77, 0.8619315), (30, 0.86164427), (121, 0.86164427), (75, 0.8575958), (8, 0.8554299), (139, 0.8554299), (182, 0.8554299), (24, 0.85349596), (74, 0.85349596), (102, 0.85349596), (38, 0.85247755), (129, 0.85247755), (131, 0.85247755), (56, 0.84964734), (98, 0.84964734), (27, 0.8495891), (100, 0.8495891), (168, 0.8495891), (68, 0.8487197), (23, 0.8485813), (104, 0.8485813), (169, 0.8485813), (89, 0.84811497), (92, 0.84811497), (144, 0.84811497), (35, 0.84453857), (107, 0.84453857), (170, 0.84453857), (48, 0.8438397), (61, 0.8424947), (152, 0.8424947), (184, 0.8424947), (32, 0.84034246), (132, 0.8403424

In [ ]:
cluster_df.head()

,Unnamed: 0,original_index,q&a,q_fa_id,question,clusters,row_num
0,2181,2181,در مورد ارزش کشته‌شدن عمرو بن عبدود توسط امام ...,_9k9An8B7wsuAN6fydoH,در مورد ارزش کشته‌شدن عمرو بن عبدود توسط امام ...,1,0
1,2349,2349,"حسن بن علی بن ابی حمزه بطائنی کیست؟,\r\tپاسخ ب...",GNlDAn8B7wsuAN6fOuT8,حسن بن علی بن ابی حمزه بطائنی کیست؟,1,1
2,2424,2424,تاریخ عقد و ازدواج حضرت علی(ع) و فاطمه زهرا(س)...,D9lFAn8B7wsuAN6fD-bO,تاریخ عقد و ازدواج حضرت علی(ع) و فاطمه زهرا(س)...,1,2
3,2459,2459,چرا پیامبر اکرم(ص) از برشمردن انساب خویش قبل ا...,ntlFAn8B7wsuAN6fzOZV,چرا پیامبر اکرم(ص) از برشمردن انساب خویش قبل ا...,1,3
4,5595,5595,نگذاشتن اسم پیامبر صلی الله علیه و آله و سلم ب...,8tkNAn8B7wsuAN6fgIO_,نگذاشتن اسم پیامبر صلی الله علیه و آله و سلم ب...,1,4


In [ ]:
cluster_df['row_num']

0        0
1        1
2        2
3        3
4        4
      ... 
183    183
184    184
185    185
186    186
187    187
Name: row_num, Length: 188, dtype: int64

tst

In [ ]:
def corpus_embedding(corpus):
  corpus_embeddings = embedder.encode(corpus, show_progress_bar=True)
  return corpus_embeddings

In [ ]:
#find similar centroids of clusters to full question but for now only one cluster(the most similar one)
#first find Euclidean distance of input to centroids using NumPy then return index of the min distance as a cluster
def find_full_question_clusters(full_question_embedding):
  pickle_in=open('centers.pickle','rb')
  centroid=pickle.load(pickle_in)
  mini=[]
  for i in centroid:
    dist = np.linalg.norm(i - full_question_embedding)
    mini.append(dist)
  cl=min(mini)
  cluster=mini.index(cl)
  return cluster


In [ ]:
def get_title_from_index(index,df):
    return df[df.index == index]["question"].values[0]

In [ ]:
#find most similar question in similar clusters but for now find for only one cluster
def find_most_similar_question_in_cluster(cluster_id,full_question_embedding):
  name=str(cluster_id)
  cluster_df = pd.read_csv('{}.csv'.format(name))

  pickle_in=open('{}.pickle'.format(name),'rb')
  u=pickle.load(pickle_in)
  u = np.append(u, full_question_embedding, axis = 0)
  cosine_sim = cosine_similarity(u)
  question_index=len(cluster_df)
  similar_questions = list(enumerate(cosine_sim[question_index]))
  sorted_similar_questions = sorted(similar_questions, key=lambda x:x[1], reverse=True)
  sorted_similar_questions.pop(0)
  result=[]
  i=0
  for question in sorted_similar_questions:
    q=get_title_from_index(question[0],cluster_df)
    print(q)

    result.append(q)

    i=i+1
    if i>15:
        break
  uniq = list(set(result))
  return uniq

In [ ]:
def recommended_questions(full_question_str):
  full_question=[]
  full_question.append(full_question_str)
  embedding=corpus_embedding(full_question)
  cluster_of_input_question=find_full_question_clusters(embedding)
  recommended_list=find_most_similar_question_in_cluster(cluster_of_input_question,embedding)
  listToStr = ' '.join(map(str, recommended_list))
  return listToStr


In [ ]:
a=recommended_questions('دادن خمس سهام اهدایی به فرزند در زمان کودکی')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

پدرم زمینی را به همسر اولش هبه کرد و پس از فوت او این زمین را به نام من سند زد که از همسر دوم ایشان می باشم. آیا این عمل مشکل شرعی دارد؟  
پدرم درباره مصرف یک سوم از اموالش وصیتی داشت و مادرم می‌گوید من وصی پدر هستم، آیا من می‌توانم سهم خود را از ثلث مطالبه کرده و با تشخیص خود صرف امور خیر کنم؟
پدر ما دارای یک قطعه زمین بود که در زمان حیاتش به سه قسمت تقسیم شده بود و با رضایت خودش دو قسمت از آن سه قسمت توسط دو نفر از پسران ایشان ساخته شد که در آن زندگی می کردند.می خواهم بدانم که آیا در تقسیم میراث متوفی ، هر سه قطعه زمین باید محاسبه شود یا فقط همان یک قسمت باقیمانده باید طبق شرع بین دختران و پسران تقسیم شود؟
در تصادفی که پدرم راننده و مقصر بود مادرم فوت کرد. دیه‌ای که بیمه خواهد داد به چه کسانی می‌رسد؟
پدرم چند سال پیش فوت کرد و در وصیت نامه خود مقدار دو میلیون تومان را جهت تهیه جهیزیه برای هر یک از دخترها در نظر گرفت. اکنون بعد از گذشت 13 سال اقدام به تقسیم اموال شده است؟ آیا  این پول به نرخ امروز در نظر گرفته می‌شود؟
پدرم چند سال پیش فوت کرد و در وصیت نامه خود مقدار دو میلیون تومان ر

In [ ]:
print(a)

{0: 'بنده از طرف دانشگاه مأمور شدم تا کارت هدیه\u200cهایی را به دانشجویان تحویل دهم، اما بعد از مدتی متوجه شدم تعدادی از کارتها مفقود شده است. اکنون آیا نسبت به آنها مدیونم؟', 1: 'پدر ما دارای یک قطعه زمین بود که در زمان حیاتش به سه قسمت تقسیم شده بود و با رضایت خودش دو قسمت از آن سه قسمت توسط دو نفر از پسران ایشان ساخته شد که در آن زندگی می کردند.می خواهم بدانم که آیا در تقسیم میراث متوفی ، هر سه قطعه زمین باید محاسبه شود یا فقط همان یک قسمت باقیمانده باید طبق شرع بین دختران و پسران تقسیم شود؟', 2: 'پدرم درباره مصرف یک سوم از اموالش وصیتی داشت و مادرم می\u200cگوید من وصی پدر هستم، آیا من می\u200cتوانم سهم خود را از ثلث مطالبه کرده و با تشخیص خود صرف امور خیر کنم؟', 3: 'در مغازه\u200cای، بدون تعیین حقوق مشغول به کار شدم. در پایان، صاحب کار، خیلی پایین\u200cتر از آنچه تصور می\u200cکردم به من دستمزد داد به همین خاطر، گوشی صاحب\u200cکارم را برداشتم. بعد از مدتی دوباره برای کار پیش همان شخص رفتم و او از موضوع مطلع شد و حقوق چند روزی که کار کرده بودم را نداد. حال تکلیف من چیست؟', 4: 'در تصا

In [ ]:
for i in range(len(a)):
  a[i]=a[i].replace('\u200c', ' ')



In [ ]:
print(a)

{0: 'بنده از طرف دانشگاه مأمور شدم تا کارت هدیه هایی را به دانشجویان تحویل دهم، اما بعد از مدتی متوجه شدم تعدادی از کارتها مفقود شده است. اکنون آیا نسبت به آنها مدیونم؟', 1: 'پدر ما دارای یک قطعه زمین بود که در زمان حیاتش به سه قسمت تقسیم شده بود و با رضایت خودش دو قسمت از آن سه قسمت توسط دو نفر از پسران ایشان ساخته شد که در آن زندگی می کردند.می خواهم بدانم که آیا در تقسیم میراث متوفی ، هر سه قطعه زمین باید محاسبه شود یا فقط همان یک قسمت باقیمانده باید طبق شرع بین دختران و پسران تقسیم شود؟', 2: 'پدرم درباره مصرف یک سوم از اموالش وصیتی داشت و مادرم می گوید من وصی پدر هستم، آیا من می توانم سهم خود را از ثلث مطالبه کرده و با تشخیص خود صرف امور خیر کنم؟', 3: 'در مغازه ای، بدون تعیین حقوق مشغول به کار شدم. در پایان، صاحب کار، خیلی پایین تر از آنچه تصور می کردم به من دستمزد داد به همین خاطر، گوشی صاحب کارم را برداشتم. بعد از مدتی دوباره برای کار پیش همان شخص رفتم و او از موضوع مطلع شد و حقوق چند روزی که کار کرده بودم را نداد. حال تکلیف من چیست؟', 4: 'در تصادفی که پدرم راننده و مقصر بود مادرم

In [ ]:
print(a[0])

بنده از طرف دانشگاه مأمور شدم تا کارت هدیه‌هایی را به دانشجویان تحویل دهم، اما بعد از مدتی متوجه شدم تعدادی از کارتها مفقود شده است. اکنون آیا نسبت به آنها مدیونم؟


In [ ]:
import fastapi
import nest_asyncio
import uvicorn
from pyngrok import ngrok

api=fastapi.FastAPI()
@api.get('/api/calculate')
def recommended_questions(full_question_str):
  full_question=[]
  full_question.append(full_question_str)
  embedding=corpus_embedding(full_question)
  cluster_of_input_question=find_full_question_clusters(embedding)
  recommended_list=find_most_similar_question_in_cluster(cluster_of_input_question,embedding)
  result={}
  for i in range(len(recommended_list)):
    val=recommended_list[i]
    val=val.replace('\u200c', ' ')
    result[i]=val
  # listToStr = ' '.join(map(str, recommended_list))
  # result={'value':listToStr}
  return result

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(api, port=8000)

Public URL: http://c44c-34-136-62-38.ngrok.io


INFO:     Started server process [60]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

پدرم زمینی را به همسر اولش هبه کرد و پس از فوت او این زمین را به نام من سند زد که از همسر دوم ایشان می باشم. آیا این عمل مشکل شرعی دارد؟  
پدرم درباره مصرف یک سوم از اموالش وصیتی داشت و مادرم می‌گوید من وصی پدر هستم، آیا من می‌توانم سهم خود را از ثلث مطالبه کرده و با تشخیص خود صرف امور خیر کنم؟
پدر ما دارای یک قطعه زمین بود که در زمان حیاتش به سه قسمت تقسیم شده بود و با رضایت خودش دو قسمت از آن سه قسمت توسط دو نفر از پسران ایشان ساخته شد که در آن زندگی می کردند.می خواهم بدانم که آیا در تقسیم میراث متوفی ، هر سه قطعه زمین باید محاسبه شود یا فقط همان یک قسمت باقیمانده باید طبق شرع بین دختران و پسران تقسیم شود؟
من برای کاری، مقداری طلا از خاله‌‌ام قرض گرفتم. روزی که مادرم پول طلا را برای تحویل به خاله‌ام برد. خاله‌‌ام قبول نکرد و گفت با این مبلغ نمی‌‌توان آن طلایی که داشتم را خریداری کرد. آیا باید قیمت امروز طلا را به او بر گردانیم؟
در تصادفی که پدرم راننده و مقصر بود مادرم فوت کرد. دیه‌ای که بیمه خواهد داد به چه کسانی می‌رسد؟
پدرم کنتور برق را دست‌کاری می‌کند، آیا استفاده ما از برق منزل 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

پدرم زمینی را به همسر اولش هبه کرد و پس از فوت او این زمین را به نام من سند زد که از همسر دوم ایشان می باشم. آیا این عمل مشکل شرعی دارد؟  
پدرم درباره مصرف یک سوم از اموالش وصیتی داشت و مادرم می‌گوید من وصی پدر هستم، آیا من می‌توانم سهم خود را از ثلث مطالبه کرده و با تشخیص خود صرف امور خیر کنم؟
پدر ما دارای یک قطعه زمین بود که در زمان حیاتش به سه قسمت تقسیم شده بود و با رضایت خودش دو قسمت از آن سه قسمت توسط دو نفر از پسران ایشان ساخته شد که در آن زندگی می کردند.می خواهم بدانم که آیا در تقسیم میراث متوفی ، هر سه قطعه زمین باید محاسبه شود یا فقط همان یک قسمت باقیمانده باید طبق شرع بین دختران و پسران تقسیم شود؟
من برای کاری، مقداری طلا از خاله‌‌ام قرض گرفتم. روزی که مادرم پول طلا را برای تحویل به خاله‌ام برد. خاله‌‌ام قبول نکرد و گفت با این مبلغ نمی‌‌توان آن طلایی که داشتم را خریداری کرد. آیا باید قیمت امروز طلا را به او بر گردانیم؟
در تصادفی که پدرم راننده و مقصر بود مادرم فوت کرد. دیه‌ای که بیمه خواهد داد به چه کسانی می‌رسد؟
پدرم کنتور برق را دست‌کاری می‌کند، آیا استفاده ما از برق منزل 

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [60]


In [ ]:
import fastapi
import nest_asyncio
import uvicorn
from pyngrok import ngrok
from typing import Optional

api=fastapi.FastAPI()
@api.get('/')
def index():
  body="<html>"\
  "<body style='padding: 10px;'>"\
  "<h1>Welcome to the API </h1>"\
  "<div>"\
  "try it: <a href='/api/calculate?x=9&y=11'>/api/calculate?x=9&y=11</a>"
  "</div>"\
  "</body>"\
  "</html>"
  return fastapi.responses.HTMLResponse(content=body)


@api.get('/api/calculate')
def calculate(x: int, y: int, z:Optional[int]=None):
  if z==0:
    return fastapi.responses.JSONResponse(content={"error":"error: Z cannot be zero"},status_code=400)
  value=x+y
  if z is not None:
    value/=z
  result={'x':x,'y':y,'z':z,'value': value}
  return result

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(api, port=8000)

Public URL: http://d2c1-34-86-183-71.ngrok.io


INFO:     Started server process [60]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     94.130.124.125:0 - "GET / HTTP/1.1" 200 OK
INFO:     94.130.124.125:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     94.130.124.125:0 - "GET /api/calculate?x=9&y=11 HTTP/1.1" 200 OK
INFO:     94.130.124.125:0 - "GET /api/calculate?x=8&y=9 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [60]


In [ ]:

!ngrok authtoken 26H4CIRVJgm3Hhb1gzuTmTHAumw_2g41vUkYVGe2iEVHryDYi

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
a="به جهت شغل همسرم به تازگی در شهری ساکن شدیم که معمولاً ده روز متوالی در آنجا حضور نداریم. وضعیت روزه و نماز ما در آن شهر چگونه \u200cاست

 Python is easy  to learn. 


In [ ]:
print(my_dic)

{0: 'a', 1: 'b'}
